In [ ]:
### Import Libraries.

library("CellChat")
library("Seurat")
library("dplyr")
library("RColorBrewer")
library("patchwork")
library("ggplot2")
library("qs")

In [ ]:
### Load Data.

setwd("/folder/")
Seurat <- qread("Seurat.qs")
Seurat$samples <- factor(Seurat$Sample_ID)

In [ ]:
### Initialize CellChat Object

cellchat <- createCellChat(
  object = Seurat,
  group.by = "Cluster_Column",
  assay = "RNA"
)

CellChatDB <- CellChatDB.human
cellchat@DB <- CellChatDB

In [ ]:
### Preprocessing

cellchat <- subsetData(cellchat)
cellchat <- identifyOverExpressedGenes(cellchat)
cellchat <- identifyOverExpressedInteractions(cellchat)

In [ ]:
### Denoise by Projecting onto PPI (Optional)

cellchat <- smoothData(cellchat, adj = PPI.human)
cellchat@idents <- droplevels(cellchat@idents)

In [ ]:
### Compute Communication Probabilities.

cellchat <- computeCommunProb(cellchat, raw.use = FALSE)
cellchat <- filterCommunication(cellchat, min.cells = 10)
cellchat <- computeCommunProbPathway(cellchat)
cellchat <- aggregateNet(cellchat)

groupSize <- as.numeric(table(cellchat@idents))

In [ ]:
### Network Visualization

netVisual_circle(cellchat@net$count,
                 vertex.weight = groupSize,
                 weight.scale = TRUE,
                 label.edge = FALSE,
                 title.name = "Number of interactions")

netVisual_circle(cellchat@net$weight,
                 vertex.weight = groupSize,
                 weight.scale = TRUE,
                 label.edge = FALSE,
                 title.name = "Interaction strength")

In [ ]:
### Pathway-Level Visualization.
## For Signaling Example, we will use PSAP.

netVisual_heatmap(cellchat, signaling = "PSAP", color.heatmap = "Reds")
netVisual_aggregate(cellchat, signaling = "PSAP", layout = "circle")
netVisual_aggregate(cellchat, signaling = "PSAP", layout = "hierarchy")
netVisual_aggregate(cellchat, signaling = "PSAP", layout = "chord")

In [ ]:
### Centrality & Role Analysis

cellchat <- netAnalysis_computeCentrality(cellchat)

netAnalysis_signalingRole_scatter(cellchat)
netAnalysis_signalingRole_scatter(cellchat, signaling = c("PSAP"))
netAnalysis_signalingRole_heatmap(cellchat, pattern = "outgoing")
netAnalysis_signalingRole_heatmap(cellchat, pattern = "incoming")

In [ ]:
### Detailed Ligand–Receptor Analysis

pairLR <- extractEnrichedLR(cellchat, signaling = "PSAP")
LR.show <- pairLR[1, ]

netVisual_individual(cellchat, signaling = "PSAP", pairLR.use = LR.show, layout = "circle")
netVisual_individual(cellchat, signaling = "PSAP", pairLR.use = LR.show, layout = "hierarchy")
netVisual_individual(cellchat, signaling = "PSAP", pairLR.use = LR.show, layout = "chord")

In [ ]:
# Bubble Plot for Selected Groups and Pathways.

netVisual_bubble(cellchat, signaling = c("PSAP"), remove.isolate = FALSE)

In [ ]:
### Gene Expression Plots.

plotGeneExpression(cellchat, signaling = "PSAP")
plotGeneExpression(cellchat, signaling = "PSAP", enriched.only = FALSE)